# Wolf Sighting Prediction - Data Preperation

## Preprocessing

The preprocessing pipeline transforms raw wolf sightings data into a structured, enriched dataset suitable for modeling. It includes the following steps:

1. **Data Loading**: Sightings data is retrieved from a remote database
2. **Geocoding**: Place names are matched to geographic coordinates using fuzzy string matching
3. **Spatial Binning and Clustering** Coordinates are discretized into spatial bins and KMeans clustering is applied to group sightings into regions
4. **Temporal Feature Engineering**: The month and season are extracted from the timestamp of sightings
5. **Historical Context Features**: The number of recent sightings and the time since the last sighting are calculated per feature


### Import Libraries

All the necessary libraries are imported here. They are listed in `requirements.txt` and can be installed using the following command:

```bash
pip install -r requirements.txt
```

In [ ]:
import pandas as pd
import numpy as np
import urllib.request
import json
from rapidfuzz import process
import folium
from sklearn.cluster import KMeans
import random
import ipywidgets as widgets
from dbrepo.RestClient import RestClient
from datetime import datetime, timedelta
import pickle

from common import *

### Load Data from the Database Repository

First, we need to fetch the raw sightings data from the database repository, hosted at dbrepo.tuwien.ac.at.

In [ ]:
load_data_widget = DownloadDataWidget(
    use_auth=False,
    database_id="4c9ac630-7ec5-491c-b727-0bea3224da91", # id of the database
    table_id="e0105e9b-acb6-4844-b3bf-aa59f35bf056", # id of the "Raw Sightings Data" table
)
load_data_widget.display()
load_data_widget.load_data()

### Spatial Feature Engineering

#### Geocoding

Since the raw sightings data does not contain any geospatial information, we need to geocode the place names to obtain their latitude and longitude. A file containing all towns and villages in Upper Austria (extracted from OpenStreetMap data) is provided in the `data` folder. We load this file, and attempt to match the place names in the sightings data with the town names in the provided file. This is done using the `rapidfuzz` library, which allows for approximate string matching. If a close enough match is found, we store the coordinates with the respective sighting entry. Otherwise, we remove the entry from the dataset.

In [ ]:
class GeonameDictionary:
    def __init__(self, file_path):
        self.geoname_dict = {}
        self.place_names = []
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                try:
                    entry = json.loads(line)
                    name = entry.get("name")
                    lat = entry.get("lat")
                    lon = entry.get("lon")
                    if name and lat is not None and lon is not None:
                        self.geoname_dict[name] = (lat, lon)
                except json.JSONDecodeError:
                    continue  # Skip lines that aren't valid JSON
            self.place_names = list(self.geoname_dict.keys())

    # Function to get closest match
    def lookup(self, query):
        match, score, _ = process.extractOne(query, self.place_names)
        return match, score

    # Function to get coordinates
    def get_coordinates(self, place_name):
        if place_name in self.geoname_dict:
            return self.geoname_dict[place_name]
        else:
            return None


geoname_dictionary = GeonameDictionary(geonames_path)

processed_df = load_data_widget.get_data().copy()

# Geocode each location in the DataFrame
for index, row in processed_df.iterrows():
    location = row.get("ort")
    if not location:
        processed_df.drop(index, inplace=True)
        continue
    match, score = geoname_dictionary.lookup(location)
    if score > 80:  # Adjust threshold as needed
        lat, lon = geoname_dictionary.get_coordinates(match)
        processed_df.at[index, 'lat'] = lat
        processed_df.at[index, 'lon'] = lon
        if score < 100:
            print(f"{location} -> {match} ({score})")
    else:
        # If no match found, remove data entry
        print(f"No match found for {location}, dropping entry.")
        processed_df.drop(index, inplace=True)

print("Processed DataFrame with geocoded locations")
print(f"Out of {len(load_data_widget.get_data())} entries, {len(processed_df)} were kept.")

#### Binning

We create coordinate bins that can be used as categorical features by the model. This is done by snapping the coordinates to a grid, where each cell covers 0.33° in latitude and 0.25° in longitude.

In [ ]:
processed_df['coord_bin'] = processed_df.apply(lambda row: coordinate_bin(row['lat'], row['lon']), axis=1).astype(str)

#### Clustering

Next, the sightings are clustered into geographic regions using the KMeans algorithm. Each sighting is assigned a `region_id` based on its coordinates. By default, the number of clusters is set to 10. This regional grouping provides the model with another categorical feature that should capture local patterns in wolf activity. After fitting the model, we save it to a file for later use.

In [ ]:
kmeans_model = KMeans(n_clusters=n_clusters, random_state=42)
processed_df['region_id'] = kmeans_model.fit_predict(processed_df[['lat', 'lon']].values)

# Save the model
with open(kmeans_path, 'wb') as f:
    pickle.dump(kmeans_model, f)

print("KMeans model saved to disk.")
print("Processed DataFrame with region IDs:")
print(processed_df[['lat', 'lon', 'region_id']].head())

As the spatial processing of data concludes, we visualize the sightings on an interactive map. Colors are assigned based on region IDs.

In [ ]:
map_center = [processed_df['lat'].mean(), processed_df['lon'].mean()]

preview_map = folium.Map(location=map_center, zoom_start=9)

colors = ['red', 'blue', 'green', 'purple', 'yellow', 'orange', 'darkred', 'lightblue', 'darkgreen', 'cadetblue', 'gray']

for _, row in processed_df.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        tooltip=f"{row['ort']} (Cluster {row['region_id']})",
        color=colors[int(row['region_id']) % len(colors)],
        fill=True,
        fill_opacity=0.6,
        radius=3,
    ).add_to(preview_map)
    
preview_map

### Temporal and Historic Feature Engineering

Next, we enrich the sightings data with time-based features. The date column is converted to a proper timestamp, and entries with invalid dates are removed. From the timestamp, two new features are extracted: The month, which is numerical, and the season, which is categorical.

In [ ]:
# Convert 'datum' column to datetime
processed_df['timestamp'] = processed_df['datum'].apply(date_string_to_datetime)

# Remove entries with malformed dates
processed_df = processed_df[processed_df['timestamp'].notnull()]

processed_df['month'] = processed_df['timestamp'].dt.month
processed_df['season'] = processed_df['month'].apply(season_from_month)

#### Recent Sightings in the Region

Next, we introduce a feature that counts how many wolf sightings occurred in the same region within the last 30 days before each individual event. This gives the model a sense of recent wolf activity in the area, allowing it to factor in short-term trends and patterns. The feature is stored in a new column called `recent_sightings`.

In [ ]:
processed_df['recent_sightings'] = processed_df.apply(
    lambda row: count_sightings_in_region(
        processed_df,
        row['region_id'],
        row['timestamp'] - recent_duration,
        row['timestamp']),
    axis=1
)

#### Last Sighting in the Region

Another temporal feature is added, this time counting the number of days since the last wolf sighting in the same region. If no earlier sightings exist, a high default value is used. This helps the model capture longer-term patterns of wolf presence or absence in an area.

In [ ]:
processed_df['days_since_last_sighting'] = processed_df.apply(
    lambda row: count_days_since_last_sighting(
        processed_df, row['region_id'], row['timestamp']),
    axis=1
)

This concludes the preprocessing of the data. We can save the processed data, including the geocoded coordinates, region IDs, and temporal features, to a CSV file in the `data` folder.

In [ ]:
SaveDataFrameWidget(
    df=processed_df,
    path=preprocessed_data_path,
    label="Save Processed Data"
).display()

## Event Generation

After preparing the sightings data through spatial and temporal enrichment, the next stage involves constructing a balanced dataset of events. Events are instances in space and time, labeled either as sightings or as synthetic non-sightings. The resulting event dataset enables the formulation of wolf sighting prediction as a binary classification task.

The below widget is provided for loading the previously generated sighting data and the KMeans model from the `data` folder.

In [ ]:
def set_processed_df(df):
    global processed_df
    processed_df = df

def set_kmeans_model(model):
    global kmeans_model
    kmeans_model = model

LoadDataWidget(
    path=preprocessed_data_path,
    on_load=set_processed_df,
    label="Load Processed Data"
).display()
LoadDataWidget(
    path=kmeans_path,
    on_load=set_kmeans_model,
    label="Load KMeans Model"
).display()

A new empty DataFrame is created to hold both positive (actual wolf sightings) and later negative (no sighting) events.

In [ ]:
sighting_events = pd.DataFrame(columns=[
    'is_sighting',
    'month',
    'season',
    'region_id',
    'coord_bin',
    'lat', 'lon',
    'recent_sightings',
    'days_since_last_sighting',
])

All real wolf sightings from the processed data are added to the `sighting_events` table as positive examples, labeled with `is_sighting = 1`. Each record carries the spatial, temporal, and historical features that were generated during preprocessing.

In [ ]:
for _, row in processed_df.iterrows():
    sighting_events = pd.concat([sighting_events, pd.DataFrame([{
        'is_sighting': 1,
        'month': row['month'],
        'season': row['season'],
        'region_id': row['region_id'],
        'coord_bin': row['coord_bin'],
        'lat': row['lat'], 'lon': row['lon'],
        'recent_sightings': row['recent_sightings'],
        'days_since_last_sighting': row['days_since_last_sighting']
    }])], ignore_index=True)

Next, we create synthetic negative events — moments and locations where no wolf sightings were recorded. For each year in the dataset, an equal number of negative events are generated by randomly selecting times and locations within the overall bounds of the sightings data. Each synthetic event is labeled with `is_sighting = 0`, and the same spatial, temporal, and historical features are calculated as for real sightings. This is intended to balance the dataset and should help the model learn to distinguish true sightings from typical background activity.

In [ ]:
min_lat = processed_df['lat'].min()
max_lat = processed_df['lat'].max()
min_lon = processed_df['lon'].min()
max_lon = processed_df['lon'].max()

# Ensure the 'timestamp' column is of datetime type
processed_df['timestamp'] = pd.to_datetime(processed_df['timestamp'])

def random_timestamp(year):
    start = datetime(year, 1, 1)
    end = datetime(year + 1, 1, 1) - timedelta(seconds=1)
    delta = end - start
    random_seconds = random.randint(0, int(delta.total_seconds()))
    return start + timedelta(seconds=random_seconds)

for year in processed_df['jahr'].unique():
    # Get the number of real data points for the year
    num_events = processed_df[processed_df['jahr'] == year].shape[0]
    # Generate negative events
    for _ in range(num_events):
        # Generate a random timestamp within the year
        timestamp = random_timestamp(int(year))
        month = timestamp.month
        season = season_from_month(month)

        # Generate random coordinates within the bounding box
        lat = random.uniform(min_lat, max_lat)
        lon = random.uniform(min_lon, max_lon)
        # Determine the region_id based on the coordinates
        region_id = kmeans_model.predict([[lat, lon]])[0]

        # Determine the recent sightings and days since last sighting
        recent_sightings = count_sightings_in_region(processed_df, region_id, timestamp - recent_duration, timestamp)
        days_since_last_sighting = count_days_since_last_sighting(processed_df, region_id, timestamp)

        # Append the negative event
        sighting_events = pd.concat([sighting_events, pd.DataFrame([{
            'is_sighting': 0,
            'month': month,
            'season': season,
            'region_id': region_id,
            'coord_bin': coordinate_bin(lat, lon),
            'lat': lat, 'lon': lon,
            'recent_sightings': recent_sightings,
            'days_since_last_sighting': days_since_last_sighting
        }])], ignore_index=True)


Finally, all sighting events are shuffled. This way, a test/train split can be obtained without requiring random sampling, which allows for easy subset creation in DBRepo. The final dataset is saved to the `data` folder.

In [ ]:
# Shuffle the events
sighting_events = sighting_events.sample(frac=1).reset_index(drop=True)
# Generate an ID for each event
sighting_events['id'] = range(1, len(sighting_events) + 1)

# Save the sighting events to a CSV file
sighting_events.to_csv(events_path, index=False)
print(f"Sighting events saved to {events_path}")

## Upload Data to Repository

The generated KMeans model can be uploaded to [TU Wien Research Data](https://test.researchdata.tuwien.ac.at/records/j49xf-khk42) using the widget below.

In [ ]:
widget = TuwrdUploadWidget(base_url="https://test.researchdata.tuwien.at", record_id='j49xf-khk42', label="Upload KMeans Model")
widget.add_file("data/wolf_sightings_kmeans.pkl", "wolf_sightings_kmeans.pkl")
widget.display()